# GAFF verification
This notebook is to compare a P3HT polymer typed with antechamber to one typed using antefoyer and GAFF-foyer. The file P3HT_16_typed.mol2 was typed with antechamber. The untyped structure is in P3HT_16.mol2.

In [1]:
import foyer
import mbuild as mb
import parmed as pmd
import hoomd
import numpy as np

/Users/jenny/miniconda3/envs/antefoyer/lib/python3.7/site-packages/hoomd/meta.py:26: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [2]:
compound = mb.load("P3HT_16.mol2")

pmd_structure = compound.to_parmed()
#compound.visualize().show()

/Users/jenny/miniconda3/envs/antefoyer/lib/python3.7/site-packages/mbuild/compound.py:2443: UserWarning: Guessing that "<C1 pos=(-0.0279, 0.0362,-0.0536), 0 bonds, id: 4798865808>" is element: "C"
  atom, element))
/Users/jenny/miniconda3/envs/antefoyer/lib/python3.7/site-packages/mbuild/compound.py:2443: UserWarning: Guessing that "<S5 pos=(-0.0831, 0.1616, 0.0422), 0 bonds, id: 4798852432>" is element: "S"
  atom, element))
/Users/jenny/miniconda3/envs/antefoyer/lib/python3.7/site-packages/mbuild/compound.py:2443: UserWarning: Guessing that "<H6 pos=(-0.0942,-0.0386,-0.0966), 0 bonds, id: 4838045072>" is element: "H"
  atom, element))


In [3]:
# Load GAFF
GAFF = foyer.forcefields.load_GAFF()

# Use foyer to parameterize
parameterized_structure = GAFF.apply(pmd_structure, assert_dihedral_params=False)

/Users/jenny/miniconda3/envs/antefoyer/lib/python3.7/site-packages/foyer/validator.py:132: ValidationWarning: You have empty smart definition(s)
  warn("You have empty smart definition(s)", ValidationWarning)
/Users/jenny/miniconda3/envs/antefoyer/lib/python3.7/site-packages/foyer/forcefield.py:248: UserWarning: Parameters have not been assigned to all proper dihedrals. Total system dihedrals: 1132, Parameterized dihedrals: 1068. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers.
  warnings.warn(msg)
/Users/jenny/miniconda3/envs/antefoyer/lib/python3.7/site-packages/foyer/forcefield.py:248: UserWarning: Parameters have not been assigned to all impropers. Total system impropers: 448, Parameterized impropers: 0. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers
  warnings.warn(msg)


In [4]:
# The typing doesn't get saved to the mbuild compound,
# so reading the file is a workaround
# the type will be stored in column 5

#typed_compound = mb.load("P3HT_16_typed.mol2")
#typed_structure= compound.to_parmed()

typed_atoms = []
read = False
with open("P3HT_16_typed.mol2", "r") as f:
    for i,line in enumerate(f.readlines()):
        if line[0] == "@" and not "ATOM" in line:
            read = False
        if read:
            typed_atoms.append(line.split())
        if line[0] == "@" and "ATOM" in line:
            read = True

In [5]:
# which atoms are different?
# looks like cc and cd, which we knew about.
diff_list = [
    (atom_antechamber[5], atom_foyer.type) for atom_antechamber, atom_foyer
    in zip(typed_atoms, parameterized_structure.atoms) 
    if atom_foyer.type != atom_antechamber[5]
]

print(set(diff_list))

{('cc', 'cc_r5'), ('cd', 'cc_r5')}
